# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import matplotlib.patches as mpatches

# Import Dataset

In [ ]:
dataset = pd.read_csv('../input/marketing-data-for-a-supermarket-in-united-states/supermarket_marketing/Supermarket_CustomerMembers.csv')
dataset

In [ ]:
dataset.info()

CustomerID is irrelevant, so we'll exclude the column.

In [ ]:
X = dataset.iloc[:,[3,4]].values
X

# Find Optimal Numer of Clusters (k) using Elbow Method

In [ ]:
wcss = []
for i in range(1,11):
    kmeans = KMeans(i, init='k-means++', random_state=42)
    kmeans.fit(X)    
    wcss.append(kmeans.inertia_)

In [ ]:
wcss

In [ ]:
plt.figure(figsize=(20, 10));
plt.plot(range(1,11), wcss);
plt.title('The Elbow Method');
plt.xlabel('Number of Clusters');
plt.ylabel('WCSS');
plt.show()

From the graph, it can be seen that k=5 is optimal.<br>
Therefore, there are 5 clusters namely 1, 2, 3, 4, 5.<br>
That is, there are 5 groups of customers in the mall.<br>

# Training K-Means

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42, init='k-means++', verbose=1)
y = kmeans.fit_predict(X)

In [ ]:
y

In [ ]:
dataset['Categories'] = pd.DataFrame(y)

In [ ]:
dataset

# Visualizing the Clusters

In [ ]:
dataset[dataset['Categories'] == 0]['Spending Score (1-100)']

In [ ]:
#empty dictionaries
clusters_centroids=dict()
clusters_radii= dict()

#looping over clusters and calculate Euclidian distance of 
#each point within that cluster from its centroid and 
#pick the maximum which is the radius of that cluster

for cluster in list(set(y)):
    clusters_centroids[cluster]=list(zip(kmeans.cluster_centers_[:, 0],kmeans.cluster_centers_[:,1]))[cluster]
    clusters_radii[cluster] = max([np.linalg.norm(np.subtract(i,clusters_centroids[cluster])) for i in zip(X[y == cluster, 0],X[y == cluster, 1])])

In [ ]:
plt.figure(figsize=(15, 10));
fig, ax = plt.subplots(1,figsize=(15,10));

plt.scatter(X[y ==0, 0], X[y ==0, 1],c='red', label='Cluster 1');
art = mpatches.Circle(clusters_centroids[0],clusters_radii[0], edgecolor='r',fill=False);
ax.add_patch(art);

plt.scatter(X[y ==1, 0], X[y ==1, 1],c='blue', label='Cluster 2');
art = mpatches.Circle(clusters_centroids[1],clusters_radii[1], edgecolor='b',fill=False);
ax.add_patch(art);

plt.scatter(X[y ==2, 0], X[y ==2, 1],c='green', label='Cluster 3');
art = mpatches.Circle(clusters_centroids[2],clusters_radii[2], edgecolor='g',fill=False);
ax.add_patch(art);

plt.scatter(X[y ==3, 0], X[y ==3, 1],c='cyan', label='Cluster 4');
art = mpatches.Circle(clusters_centroids[3],clusters_radii[3], edgecolor='cyan',fill=False);
ax.add_patch(art);

plt.scatter(X[y ==4, 0], X[y ==4, 1],c='magenta', label='Cluster 5');
art = mpatches.Circle(clusters_centroids[4],clusters_radii[4], edgecolor='magenta',fill=False);
ax.add_patch(art);

plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1],c='black', marker='v', s=100, label='Centroids');

plt.title('Clusters')
plt.xlabel('Annual Income (k$)');
plt.ylabel('Spending Score (1-100)');
plt.legend(['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5']);
plt.show();

# Analysis
- Cluster 3 (Green) : People with low annual income and who spends lower in the mall. These are customers still less targeted.
<br><br>
- Cluster 2 (Blue) : People with high annual income and who still doesn't spend much money in the mall. These customers can be targeted so that they can be tempted to spend more in the mall. These are the customers with 3rd priority to be targeted (1 - Cluster 5, 2 - Cluster 2).
<br><br>
- Cluster 4 (Cyan) : People with low annual income and who spend a lot in the mall. These Customers are less targeted by the mall (if the mall wishes to so that the people with low income doesn't need to spend too much in the mall - ethics).
<br><br>
- Cluster 5 (Magenta) : People with higher annual income and who spends more in the mall. These customers are likely to buy more in the mall so they are the priority targets for the mall.
<br><br>
- Cluster 1 (Red) : People with average annual income and spends averagely in the mall.